**# Import necessary libraries**

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Reshape

**# Load the dataset**

In [53]:
data = pd.read_csv('fer2013.csv')

**# Inspect the dataset**

In [54]:
print(data.head())
print(data.columns)

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
Index(['emotion', 'pixels', 'Usage'], dtype='object')


**# Update these column names based on your actual dataset**

In [55]:
context_column = 'Usage'
target_column = 'emotion'

**# Verify the column names**

In [56]:
expected_columns = [context_column, 'pixels', target_column]
for col in expected_columns:
    if col not in data.columns:
        raise ValueError(f"Expected column '{col}' not found in the dataset")

**# Convert pixels column to numpy arrays**

In [57]:
data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))

**# Separate features and target**

In [58]:
X = np.stack(data['pixels'].values)
X = X / 255.0  # Normalize pixel values
y = data[target_column]

**# Reshape X to (number of samples, 48, 48, 1) since images are 48x48 pixels**

In [59]:
X = X.reshape(-1, 48, 48, 1)

**# One-hot encode the target variable**

In [60]:
y = pd.get_dummies(y).values


**# One-hot encode the context variable**

In [73]:
context_encoder = OneHotEncoder(sparse=False)
context_encoded = context_encoder.fit_transform(data[[context_column]])

c:\Users\25dea\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


**# Flatten the image data**

In [74]:
X_flat = X.reshape(X.shape[0], -1)

**# Combine the flattened image data with the context information**

In [75]:
X_combined = np.concatenate([X_flat, context_encoded], axis=1)


**# Split the encoded data**

In [76]:
X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(X_combined, y, test_size=0.2, random_state=42)


**# Neural network model**

In [77]:
model = Sequential([
    Input(shape=(X_train_enc.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train_enc.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**# Train the model**

In [78]:
model.fit(X_train_enc, y_train_enc, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.2567 - loss: 1.8092 - val_accuracy: 0.3135 - val_loss: 1.7261
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3277 - loss: 1.6977 - val_accuracy: 0.3506 - val_loss: 1.6783
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3406 - loss: 1.6641 - val_accuracy: 0.3521 - val_loss: 1.6622
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3438 - loss: 1.6576 - val_accuracy: 0.3539 - val_loss: 1.6507
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3538 - loss: 1.6422 - val_accuracy: 0.3615 - val_loss: 1.6410
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3591 - loss: 1.6245 - val_accuracy: 0.3713 - val_loss: 1.6193
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3726 - loss: 1.6017 - val_accuracy: 0.3633 - val_loss: 1.6199
Epoch 8/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3757 - loss: 1.5983 - val_accuracy: 0.

**# Evaluate the multi-context model**

In [79]:
loss, accuracy = model.evaluate(X_test_enc, y_test_enc)
print(f'Multi-context model accuracy: {accuracy}')

225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.3485 - loss: 1.6442
Multi-context model accuracy: 0.35079410672187805
